## Process Xenium

In [1]:
import pandas as pd
import scanpy as sc
import squidpy as sq

In [2]:
from benchmark import _sample_resource

Load data 

In [3]:
# as in https://squidpy.readthedocs.io/en/stable/external_tutorials/tutorial_xenium.html
adata = sc.read_10x_h5(filename='../../data/xenium_data/cell_feature_matrix.h5')
obs = pd.read_csv('../../data/xenium_data/cells.csv')

In [4]:
obs.set_index(adata.obs_names, inplace=True)
adata.obs = obs.copy()
adata.obsm["spatial"] = adata.obs[["x_centroid", "y_centroid"]].copy().to_numpy()

Preprocess

In [5]:
sc.pp.filter_cells(adata, min_counts=10)
sc.pp.filter_genes(adata, min_cells=5)

In [8]:
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)

Compute connectivity matrix

In [9]:
sq.gr.spatial_neighbors(adata, coord_type="generic", delaunay=True)

In [10]:
sq.gr.spatial_autocorr(
    adata,
    mode="moran",
    n_perms=None,
    n_jobs=1,
)

In [11]:
top = adata.uns["moranI"].sort_values("I", ascending=False).head(30).index

In [16]:
resource = _sample_resource(adata, n_lrs=500)

In [17]:
adata.uns['resource'] = resource

In [18]:
adata.write_h5ad("xenium.h5ad")